<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="1500">

# Installation detectron2

In [ ]:
# install dependencies:
# (use +cu100 because colab is on CUDA 10.0)
# ----------------------UNCOMMENT FROM HERE----------------------------------
# !pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
# !pip install cython pyyaml==5.1
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# import torch, torchvision
# torch.__version__
# !gcc --version
# ---------------------UNCOMMENT TILL HERE-----------------------------------
# opencv is pre-installed on colab

In [ ]:
# UNCOMMENT WHEN RUNNING THIS NOTEBOOK FOR THE FIRST TIME
# !git clone https://github.com/facebookresearch/detectron2 detectron2_repo

## You can replace cu101 with "cu{100,92}" or "cpu".
# pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
# !cp detectron2_repo/configs ./ -r

Restart runtime in case you are installing detectron for the first time

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# common libraries
import numpy as np
import cv2
import random
import os
import wandb

try:
    from google.colab.patches import cv2_imshow
except:
    os.system(f"""pip install google.colab""")
    from google.colab.patches import cv2_imshow
import glob
import matplotlib.pyplot as plt
# import some common .detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode

# Train on a custom dataset

If you want to use a custom dataset while also reusing detectron2’s data loaders, you will need to

* Register your dataset (i.e., tell detectron2 how to obtain your dataset).
* Optionally, register metadata for your dataset.

## Prepare the dataset

To register your dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we write a function to parse it and prepare it into detectron2's standard format. See the tutorial for more details.


### Folder structure to follow to directly use this notebook

```
EndoCV_Det
│   class_list_bbox.txt  
│
└─── TRAIN
│      └ images
│      └ bbox
└─── VAL
│      └ images
│      └ bbox
└─── HOLDOUT  (Holdout set is optional)
       └ images
       └ bbox
```

In [ ]:
def get_bb(img_path, img_name):
    """
        This function returns annotations in the format
        xyxy_ABS: xmin, ymin, xmax, ymax - ABSOLUTE
        
        Function built taking into consideration the dataset
        provided by EAD2020
        
        change the path to the bounding box folder(if not training on EAD)
        and change the loading method(if using .mat) accordingly
    """
    # change the path to the bounding box folder(if not training on EAD)
    # and change the loading method accordingly
    bb_path = img_path[:-7]+'bbox/'
    img = plt.imread(img_path+img_name)
    m, n, _ = img.shape
    labels = np.loadtxt(bb_path+img_name[:-4]+'.txt').reshape(-1, 5)
    classes = np.empty(len(labels), dtype=np.int32)
    xyxy = np.empty((len(labels), 4), dtype=np.int32)
    for i, label in enumerate(labels):
        cls, x, y, w, h = label
        x1 = (x-w/2.)
        x2 = x1 + w
        y1 = (y-h/2.)
        y2 = y1 + h
        x1 = np.clip(int(x1 * n), 0, n-1) ; x2 = np.clip(int(x2*n), 0, n-1)
        y1 = np.clip(int(y1 * m), 0, m-1) ; y2 = np.clip(int(y2*m), 0, m-1)
        classes[i] = int(cls)
        xyxy[i] = [x1, y1, x2, y2]
        
    return xyxy, classes

def _get_dicts(phase):
    if phase == 'train':
        path = 'EndoCV_Det/TRAIN/images/'
    elif phase == 'val':
        path = 'EndoCV_Det/VAL/images/'
    else:
        raise(Exception('Provide either "Train" or "Val"'))
    
    def get_dicts():
        dataset_dicts = []
        img_list = os.listdir(path)
        for idx, i in enumerate(img_list):
            record = {}
            img = plt.imread(path+i)
            height, width, _ = img.shape        
            record["file_name"] = path+i
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            proposal_bb, proposal_logits = get_bb(path, i)
            objs=[]
            for j in range(len(proposal_bb)):
                obj = {
                    "bbox": [proposal_bb[j][0], proposal_bb[j][1], proposal_bb[j][2], proposal_bb[j][3]],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": proposal_logits[j],
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
            record["thing_classes"] = ["specularity","saturation",
                                          "artifact", "blur", "contrast", "bubbles",
                                          "instrument", "blood"]
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("endo1_" + d, _get_dicts(d))
#     if d == "train":
    MetadataCatalog.get("endo1_" + d).set(thing_classes=["specularity","saturation",
                                      "artifact", "blur", "contrast", "bubbles",
                                      "instrument", "blood"])
#     elif d == "val":
#     MetadataCatalog.get("endo1_" + d).set(thing_classes=["specularity","saturation",
#                                       "artifact", "blur", "contrast", "bubbles",
#                                       "instrument", "blood"],
#                                           pred_classes=["specularity","saturation",
#                                       "artifact", "blur", "contrast", "bubbles",
#                                       "instrument", "blood"])
endo_metadata = MetadataCatalog.get("endo1_train") 

## Visualisation



In [ ]:
# Uncomment if you want to visualize training data

# dataset_dicts = _get_dicts('train')()
# for d in random.sample(dataset_dicts, 3):
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=endo_metadata, scale=0.5)
#     vis = visualizer.draw_dataset_dict(d)
#     cv2_imshow(vis.get_image()[:, :, ::-1])
    

### To choose model:
    detectron2_repo-->configs
    Accordingly choose Detection or whatever you require
    For eg:
        "detectron2_repo/configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml"
### To get weights:
    detectron2_repo-->detectron2-->model_zoo-->model_zoo.py
    For eg: copy any link
        "COCO-Detection/faster_rcnn_R_50_C4_3x.yaml": "137849393/model_final_f97cb7.pkl"
    delete the mid part and add detectron2:// to the start
        "detectron2://COCO-Detection/faster_rcnn_R_50_C4_3x/137849393/model_final_f97cb7.pkl"
    use this format as a link to get the model weights

# Train

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("endo1_train",)
cfg.DATASETS.TEST = ("endo1_val",)   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/retinanet_R_50_FPN_3x/137849486/model_final_4cafe0.pkl"  # initialize from model zoo

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.RETINANET.NUM_CLASSES = 8

In [ ]:
# Automatically switch to a new tensorboard directory every time this cell is executed
i = 1
while os.path.exists(os.path.join(cfg.OUTPUT_DIR, f'run{i}')):
    i += 1

cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
cfg.OUTPUT_DIR = os.path.join(cfg.OUTPUT_DIR, f'run{i}')
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Create trainer
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

# WandB
config = dict(cfg)
del config['MODEL']
wandb.init(
    name=f"Run {i}",  # Name to display in wandb
#     notes=f"{i}th run.",  # Notes, if any
    notes=f"1st real run.",  # Notes, if any
    project='endocv-det',  # Project name as on WandB website
    sync_tensorboard=True,
    config=config  # Save hyperparameters
)
# wandb.watch(trainer.model)

# Launch Tensorboard
%load_ext tensorboard
!pkill tensorboard
%tensorboard --logdir=output/run{i} --port=6007

# Commence Training
trainer.train()

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("endo1_val", )
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode

for t in ['train', 'val']:
    dataset_dicts = _get_dicts(t)()

    print("\n"*2 + t.upper() + "\n"*2)
    for d in random.sample(dataset_dicts, 10):
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=MetadataCatalog.get(f"endo1_{t}"), 
                       scale=0.8, 
    #                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2_imshow(v.get_image()[:, :, ::-1])
        os.makedirs(os.path.join(cfg.OUTPUT_DIR, f'{t}_preds/'), exist_ok=True)
        
        # WandB
        wandb.log({f"{t}: {d['file_name']}": [wandb.Image(v.get_image(), caption=f"{d['file_name']}")]})

We can also evaluate its performance using AP metric implemented in COCO API.
This gives an AP of ~70%. Not bad!

# &lt;The following still remains to be changed>


In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("balloon_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "balloon_val")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

# Other types of builtin models

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(v.get_image()[:, :, ::-1])

# Run panoptic segmentation on a video

In [ ]:
# This is the video we're going to process
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("ll8TgCZ0plk", width=500)
display(video)

In [ ]:
# Install dependencies, download the video, and crop 5 seconds for processing
!pip install youtube-dl
!pip uninstall -y opencv-python opencv-contrib-python
!apt install python3-opencv  # the one pre-installed have some issues
!youtube-dl https://www.youtube.com/watch?v=ll8TgCZ0plk -f 22 -o video.mp4
!ffmpeg -i video.mp4 -t 00:00:06 -c:v copy video-clip.mp4

In [ ]:
# Run frame-by-frame inference demo on this video (takes 3-4 minutes)
# Using a model trained on COCO dataset
!cd detectron2_repo && python demo/demo.py --config-file configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml --video-input ../video-clip.mp4 --confidence-threshold 0.6 --output ../video-output.mkv \
  --opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl

In [ ]:
# Download the results
from google.colab import files
files.download('video-output.mkv')